In [5]:
import numpy as np
import pandas as pd
import

In [44]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import SimpleImputer
from sklearn.impute import KNNImputer

# Convert data to pandas DataFrame

In [3]:
l = ['010011','110111','101110','011101','011101','001101','?11?0?','?11?2?']

In [11]:
s = pd.Series(l)

In [12]:
s

0    010011
1    110111
2    101110
3    011101
4    011101
5    001101
6    ?11?0?
7    ?11?2?
dtype: object

In [18]:
df = s.str.split(pat='',n=6, expand=True)
df

,0,1,2,3,4,5,6
0,,0,1,0,0,1,1
1,,1,1,0,1,1,1
2,,1,0,1,1,1,0
3,,0,1,1,1,0,1
4,,0,1,1,1,0,1
5,,0,0,1,1,0,1
6,,?,1,1,?,0,?
7,,?,1,1,?,2,?


In [20]:
df = df.replace({'?': np.nan})
df

,0,1,2,3,4,5,6
0,,0,1,0,0,1,1
1,,1,1,0,1,1,1
2,,1,0,1,1,1,0
3,,0,1,1,1,0,1
4,,0,1,1,1,0,1
5,,0,0,1,1,0,1
6,,NaN,1,1,NaN,0,NaN
7,,NaN,1,1,NaN,2,NaN


In [24]:
df = df.drop(columns=[0])
df

KeyError: '[0] not found in axis'

In [29]:
df2 = df.astype(int, errors='ignore', copy=True)
df2

,1,2,3,4,5,6
0,0,1,0,0,1,1
1,1,1,0,1,1,1
2,1,0,1,1,1,0
3,0,1,1,1,0,1
4,0,1,1,1,0,1
5,0,0,1,1,0,1
6,NaN,1,1,NaN,0,NaN
7,NaN,1,1,NaN,2,NaN


In [50]:
df3 = df2.transpose()
df3

,0,1,2,3,4,5,6,7
1,0,1,1,0,0,0,NaN,NaN
2,1,1,0,1,1,0,1,1
3,0,0,1,1,1,1,1,1
4,0,1,1,1,1,1,NaN,NaN
5,1,1,1,0,0,0,0,2
6,1,1,0,1,1,1,NaN,NaN


In [58]:
df4 = df2.copy(deep=True)

In [59]:
df4.loc[6.5] = [np.nan,np.nan,np.nan,np.nan,np.nan,np.nan]
df4.loc[7.5] = [np.nan,np.nan,np.nan,np.nan,np.nan,np.nan]

In [63]:
df4 = df4.sort_index().reset_index(drop=True)
df4

,1,2,3,4,5,6
0,0,1,0,0,1,1
1,1,1,0,1,1,1
2,1,0,1,1,1,0
3,0,1,1,1,0,1
4,0,1,1,1,0,1
5,0,0,1,1,0,1
6,NaN,1,1,NaN,0,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,1,1,NaN,2,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN


# Imputation with Scikit-Learn

## nearest neighbors imputation: not working

In [39]:
# weights is uniform or distance
imputer = KNNImputer(n_neighbors=2, weights='distance')

In [41]:
imputer.fit_transform(df2)

array([[0. , 1. , 0. , 0. , 1. , 1. ],
       [1. , 1. , 0. , 1. , 1. , 1. ],
       [1. , 0. , 1. , 1. , 1. , 0. ],
       [0. , 1. , 1. , 1. , 0. , 1. ],
       [0. , 1. , 1. , 1. , 0. , 1. ],
       [0. , 0. , 1. , 1. , 0. , 1. ],
       [0. , 1. , 1. , 1. , 0. , 1. ],
       [0.5, 1. , 1. , 0.5, 2. , 1. ]])

## Univariate imputation: mean, median or most_frequent not working

In [48]:
imp = SimpleImputer(missing_values=np.nan, strategy='most_frequent')

In [52]:
imp.fit_transform(df2)

array([['0', '1', '0', '0', '1', '1'],
       ['1', '1', '0', '1', '1', '1'],
       ['1', '0', '1', '1', '1', '0'],
       ['0', '1', '1', '1', '0', '1'],
       ['0', '1', '1', '1', '0', '1'],
       ['0', '0', '1', '1', '0', '1'],
       ['0', '1', '1', '1', '0', '1'],
       ['0', '1', '1', '1', '2', '1']], dtype=object)

## Multivariate imputation

In [53]:
imp = IterativeImputer(max_iter=10, random_state=0)

In [64]:
imp.fit_transform(df4)

array([[ 0.        ,  1.        ,  0.        ,  0.        ,  1.        ,
         1.        ],
       [ 1.        ,  1.        ,  0.        ,  1.        ,  1.        ,
         1.        ],
       [ 1.        ,  0.        ,  1.        ,  1.        ,  1.        ,
         0.        ],
       [ 0.        ,  1.        ,  1.        ,  1.        ,  0.        ,
         1.        ],
       [ 0.        ,  1.        ,  1.        ,  1.        ,  0.        ,
         1.        ],
       [ 0.        ,  0.        ,  1.        ,  1.        ,  0.        ,
         1.        ],
       [-0.16664873,  1.        ,  1.        ,  0.8333444 ,  0.        ,
         0.99999958],
       [ 0.45832332,  0.75      ,  0.75      ,  0.8333355 ,  0.625     ,
         0.62500653],
       [ 1.83323526,  1.        ,  1.        ,  0.8333396 ,  2.        ,
        -0.99994735],
       [ 0.45832332,  0.75      ,  0.75      ,  0.8333355 ,  0.625     ,
         0.62500653]])